In [4]:
import ee
ee.Authenticate()


ModuleNotFoundError: No module named '_curses'

In [ ]:
import requests
import os
import pandas as pd
import numpy as np

# Initialize Earth Engine
ee.Initialize(project='ee-nguyendangkhoi9517')

# Read data from CSV
df = pd.read_csv('/Data_csv/flood_data.csv')

# Define color palettes
durationPalette = ['c3effe', '1341e8', '051cb0', '001133']
elevationVis = {
    'min': 0.0,
    'max': 300.0,  # Adjust based on elevation range of your area
    'palette': ['0000ff', '00ffff', 'ffff00', 'ff0000', 'ffffff']
}

# Define square size (10 km² -> 10,000 meters)
square_size_m = 10000  # each side of the square

# Loop through each row in the DataFrame
for index, row in df.iterrows():
    center_lon = row['dfo_centroid_x']
    center_lat = row['dfo_centroid_y']
    area_km2 = row['gfd_area']  # Get area from column

    # Calculate the radius of the area of interest in meters
    radius = np.sqrt(area_km2) * 1000 / 2  # Radius in meters

    # Skip if radius is zero or very close to zero
    if radius <= 0:
        print(f"Skipping event {index} due to zero radius.")
        continue

    # Parse and format the event's start and end dates
    try:
        began_date_raw = row['dfo_began']
        ended_date_raw = row['dfo_ended']

        # Convert two-digit years if necessary
        if len(began_date_raw.split('/')[-1]) == 2:
            began_date_raw = began_date_raw[:-2] + '20' + began_date_raw[-2:]
        if len(ended_date_raw.split('/')[-1]) == 2:
            ended_date_raw = ended_date_raw[:-2] + '20' + ended_date_raw[-2:]

        began_date = pd.to_datetime(began_date_raw, format='%m/%d/%Y', errors='coerce')
        ended_date = pd.to_datetime(ended_date_raw, format='%m/%d/%Y', errors='coerce')

        if pd.isna(began_date) or pd.isna(ended_date):
            print(f"Invalid date format for event {index}. Skipping this event.")
            continue

        # Format dates to strings
        began_date_str = began_date.strftime('%Y-%m-%d')
        ended_date_str = ended_date.strftime('%Y-%m-%d')
    except Exception as e:
        print(f"Error parsing dates for event {index}: {e}")
        continue

    # Create the central point and circular area around it
    point = ee.Geometry.Point(center_lon, center_lat)
    circle = point.buffer(radius)

    # Define the number of squares along each axis to cover the circle
    num_squares = int(np.ceil(radius / square_size_m))

    # Generate grid squares around the central point
    squares = []
    for i in range(-num_squares, num_squares + 1):
        for j in range(-num_squares, num_squares + 1):
            # Calculate the bounds for each 10 km² square
            square = ee.Geometry.Rectangle([
                center_lon + i * square_size_m / 111320 - square_size_m / 111320 / 2,
                center_lat + j * square_size_m / 111320 - square_size_m / 111320 / 2,
                center_lon + i * square_size_m / 111320 + square_size_m / 111320 / 2,
                center_lat + j * square_size_m / 111320 + square_size_m / 111320 / 2
            ])
            # Only add squares that intersect with the circle
            if circle.intersects(square):
                squares.append(square)

    print(f"Number of squares for event {index}: {len(squares)}")

    # Get flood data from Global Flood Database within the circular region
    flood_events = ee.ImageCollection('GLOBAL_FLOOD_DB/MODIS_EVENTS/V1').filterBounds(circle)

    # Get elevation data from Copernicus DEM GLO-30
    dem = ee.ImageCollection('COPERNICUS/DEM/GLO30').select('DEM').mosaic()

    # Loop through each square and download images for both flood duration and elevation
    for square_index, square in enumerate(squares):
        # Filter flood events by date range
        flood_events_filtered = flood_events.filterDate(began_date_str, ended_date_str)

        if flood_events_filtered.size().getInfo() > 0:
            flood_image = flood_events_filtered.first()
            if flood_image:
                # Visualize flood duration layer
                duration_visualized = flood_image.select('duration').visualize(min=0, max=10, palette=durationPalette)

                # Calculate center of the square for filename
                square_center = square.centroid().coordinates().getInfo()
                square_center_lat = square_center[1]
                square_center_lon = square_center[0]

                # Generate thumbnail URL for flood duration
                thumbnail_params = {
                    'region': square,
                    'dimensions': 512,
                    'format': 'png'
                }
                url = duration_visualized.getThumbURL(thumbnail_params)

                # Define the save path for flood duration image
                save_path_flood = f"/Data_img/flood_maps/flood_map_{began_date_str}_{ended_date_str}_{square_center_lat}_{square_center_lon}.png"
                os.makedirs(os.path.dirname(save_path_flood), exist_ok=True)

                # Download and save the flood duration image
                response = requests.get(url)
                if response.status_code == 200:
                    with open(save_path_flood, 'wb') as f:
                        f.write(response.content)
                    print(f"Flood map for square {square_index} of event {index} saved to: {save_path_flood}")
                else:
                    print(f"Failed to download flood map for square {square_index} of event {index}")
        else:
            print(f"No flood events for event {index}")

        # Clip DEM to the square region for elevation data
        dem_square = dem.clip(square)

        # Visualize the clipped DEM data
        dem_visualized = dem_square.visualize(**elevationVis)

        # Generate thumbnail URL for the elevation map
        url_dem = dem_visualized.getThumbURL(thumbnail_params)

        # Define the save path for elevation image
        save_path_dem = f"/Data_img/elevation_maps/elevation_map_{began_date_str}_{ended_date_str}_{square_center_lat}_{square_center_lon}.png"
        os.makedirs(os.path.dirname(save_path_dem), exist_ok=True)

        # Download and save the elevation image
        response_dem = requests.get(url_dem)
        if response_dem.status_code == 200:
            with open(save_path_dem, 'wb') as f:
                f.write(response_dem.content)
            print(f"Elevation map for square {square_index} of event {index} saved to: {save_path_dem}")
        else:
            print(f"Failed to download elevation map for square {square_index} of event {index}")
